In [ ]:
## sync version - slow but stable
import re
import pandas as pd
import urllib.parse
from datetime import datetime
import requests as rq
import sqlite3
from tenacity import retry, stop_after_attempt, wait_exponential, RetryError
from tqdm import tqdm

BASE="https://www.youswear.com/index.asp"

def before_sleep_callback(retry_state):
    print(f"Attempt {retry_state.attempt_number} failed. Retrying...")
    if retry_state.outcome.exception():
        print(f"  Error: {retry_state.outcome.exception()}")

retry_strategy = retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=1, min=1, max=10),
    before_sleep=before_sleep_callback
)

@retry_strategy
def get_with_retry(url):
    return rq.get(url)


languages : list = re.findall( r'class="list-group-item">(.*?)</a></li>', get_with_retry(BASE).text)
df = pd.DataFrame(columns=["language", "phrase", "meaning", "voteup", "votedown"])
for lang in tqdm(languages):
    try:
        html = get_with_retry(BASE + "?language=" + urllib.parse.quote_plus(lang)).text
    except Exception as e:
        print("Error getting ?language="+lang)
        continue
    try:
        rows = re.findall(r'<tr>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+<td>.*?votefor.*?</a>.*?(\d+).*?<a.*?voteagainst.*?</a>.*?(\d+).*?</span>\s+</td>', html,re.S)
    except Exception as e:
        print("Error parsing ?language="+lang)
        continue
    rows = [{"language": lang, "phrase": row[0], "meaning": row[1], "voteup": row[2], "votedown": row[3]} for row in rows]
    df = pd.concat([df, pd.DataFrame(rows)])

df.to_csv("youswear.csv", index=False)
df_csv = df.copy()

# db_file = "youswear.sqlite.db"
# conn = sqlite3.connect(db_file)
# df.to_sql("swear", conn, if_exists="replace", index=False)
# conn.commit()
# conn.close()

languages_df = df[["language"]].drop_duplicates().reset_index(drop=True)
languages_df["language_id"] = languages_df.index + 1
language_id_mapping = languages_df.set_index("language")["language_id"].to_dict()
df["language_id"] = df["language"].map(language_id_mapping)
df = df.drop(columns=["language"])
# df = df.rename(columns={"language_id":"language"})
df = df[["language_id", "phrase", "meaning", "voteup", "votedown"]]

db_file = "youswear.sqlite.db"
conn = sqlite3.connect(db_file)

conn.executescript('''
    PRAGMA foreign_keys = ON;
    CREATE TABLE IF NOT EXISTS languages (
        language_id INTEGER PRIMARY KEY,
        language TEXT UNIQUE
    );
    CREATE TABLE IF NOT EXISTS swear (
        language_id INTEGER,
        phrase TEXT,
        meaning TEXT,
        voteup INTEGER,
        votedown INTEGER,
        FOREIGN KEY (language_id) REFERENCES languages(language_id)
    );
''')
languages_df.to_sql("languages", conn, if_exists="replace", index=False)
df.to_sql("swear", conn, if_exists="replace", index=False)
conn.commit()
conn.close()

100%|██████████| 904/904 [25:27<00:00,  1.69s/it]


In [ ]:
## adds / update the column "region" in table "languages" by LLM classifier
import json
import logging
import os
import sqlite3
from getpass import getpass

import requests as rq
from tenacity import retry, stop_after_attempt, wait_exponential, before_sleep_log, RetryError
from tqdm import tqdm

# API Configuration
BASE = os.environ.get("OPENAI_API_BASE") or getpass("Input API base like https://api.example/v1: ") #"https://api.example/v1"
KEY = "sk-no-need"
MODEL = "THUDM/glm-4-9b-chat"

# Configure logging for tenacity
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def before_sleep_callback(retry_state):
    logger.warning(f"Attempt {retry_state.attempt_number} failed. Retrying...")
    if retry_state.outcome.exception():
        logger.warning(f"  Error: {retry_state.outcome.exception()}")

retry_strategy = retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=1, min=1, max=10),
    before_sleep=before_sleep_callback
)

@retry_strategy
def get_llm_response(messages):
    res = rq.post(
        BASE + "/chat/completions",
        headers={"Authorization": "Bearer " + KEY, "Content-Type": "application/json"},
        json={"model": MODEL, "messages": messages},
        timeout=30
    )
    res.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    return res.json()

def extract_json_string(s, b='{', e='}'):
    try:
        i,j = s.rfind(b.strip()), s.rfind(e.strip())
        if i == -1 or j == -1 or i >= j:
            raise ValueError("No valid JSON object found")
        return s[i:j + len(e.strip())]
    except ValueError as e:
        raise ValueError(f"Error extracting JSON: {e} from string: {s}")

@retry(stop=stop_after_attempt(5), before_sleep=before_sleep_log(logger, logging.WARNING))
def get_regions_for_languages(languages):
    messages = [
        {"role": "system", "content": "Assistant is a classifier that returns in specified JSON format."},
        {"role": "user", "content": (
        "Classify the following languages into their corresponding UN geographical subregions. "
        "Return a JSON object where keys are the language names and values are the subregion names. "
        "If a language does not have a corresponding UN region (e.g., it's fictional, it's animal, etc.), use `null` as the value.\n\n"
        "<example>\n"
        'Input: ["Yoruba","Guoyu","Inuktitut","Klingon","Goose","Turkey","Alien"]\n'
        'Output: {"Yoruba":"Western Africa","Guoyu":"Eastern Asia","Inuktitut":"Northern America","Klingon":null,"Goose":null,"Turkey":"Western Asia","Alien":null}\n</example>\n'
        f'Input: {json.dumps(languages)}\n'
        "Output:"
        )},
    ]

    response = get_llm_response(messages)
    response_content = response["choices"][0]["message"]["content"]
    json_string = extract_json_string(response_content)
    return json.loads(json_string)

##  -- main --

db_file = "youswear.sqlite.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

try:
    # Get all languages
    cursor.execute("SELECT language FROM languages")
    languages = [row[0] for row in cursor.fetchall()]

    # Group languages
    group_size = 20
    language_groups = [
        languages[i:i + group_size] for i in range(0, len(languages), group_size)
    ]

    # Check if the 'region' column exists, add if not
    cursor.execute("PRAGMA table_info(languages)")
    columns = [col[1] for col in cursor.fetchall()]
    if "region" not in columns:
        cursor.execute("ALTER TABLE languages ADD COLUMN region TEXT")
        conn.commit()

    # Get regions and update database
    for group in tqdm(language_groups, desc="Get regions and update database"):
        try:
            regions_map = get_regions_for_languages(group)
            for language, region in regions_map.items():
                if region is not None:  # update only if region != NULL
                    cursor.execute(
                        "UPDATE languages SET region = ? WHERE language = ?",
                        (region, language)
                    )
                else:
                    logger.info(f"Language '{language}' not mapped to a region.")
            conn.commit()
        except RetryError as e:
            logger.error(f"Failed to process group after multiple retries: {group}. Error: {e}")
            continue # Continue to next group
        except (ValueError, KeyError, rq.RequestException) as e:
            logger.error(f"Error processing group {group}: {e}")
            continue  # Continue processing other groups even if one fails.

    cursor.execute("UPDATE languages SET region = NULL WHERE LOWER(region) IN ('undefined', 'null')")
    conn.commit()

except sqlite3.Error as e:
    print(f"Database error: {e}")
finally:
    conn.close()

Input API base like https://api.example/v1: ··········


Get regions and update database: 100%|██████████| 45/45 [03:13<00:00,  4.30s/it]


In [ ]:
df_csv.head()

,language,phrase,meaning,voteup,votedown
0,(kwalian dialect),Jeben ti boga magarac,May God fuck your donkey,1,1
0,(pashto),da khar bachiya,son of donkey,10,1
1,(pashto),Da spi zou,Son of a bitch,10,4
2,(pashto),De gheen sar,Dick head,0,0
3,(pashto),Dere rabande grana ye,I hate you so much,1,11


In [ ]:
## sql playground
db_file = "youswear.sqlite.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(languages)")
# print(cursor.fetchall())
columns = [col[1] for col in cursor.fetchall()]
print(columns)
conn.commit()
conn.close()

In [ ]:
## async version - too many HTTP 5xx errors don't use
import re
import pandas as pd
import urllib.parse
from datetime import datetime
import asyncio
import httpx

BASE = "https://www.youswear.com/index.asp"
MAX_CONCURRENCY = 5
semaphore = asyncio.Semaphore(MAX_CONCURRENCY)

async def _web_scrape(lang: str, client: httpx.AsyncClient):
    """Fetches and processes data for a single language."""
    async with semaphore:
        try:
            url = BASE + "?language=" + urllib.parse.quote_plus(lang)
            response = await client.get(url)
            response.raise_for_status()  # Raise HTTPStatusError for bad responses (4xx or 5xx)
            html = response.text
        except httpx.RequestError as e:
            print(f"Error getting ?language={lang}: {e}")
            return []
        except httpx.HTTPStatusError as e:
            print(f"HTTP error for ?language={lang}: {e}")
            return []
        except Exception as e:
            print(f"General Error on request ?language={lang}: {e}")
            return []

        try:
            rows = re.findall(r'<tr>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+<td>.*?votefor.*?</a>.*?(\d+).*?<a.*?voteagainst.*?</a>.*?(\d+).*?</span>\s+</td>', html, re.S)
            return [{"language": lang, "phrase": row[0], "meaning": row[1], "voteup": row[2], "votedown": row[3]} for row in rows]
        except Exception as e:
            print(f"Error parsing ?language={lang}: {e}")
            return []


async def web_to_df():
    """Fetches data for all languages concurrently and creates a DataFrame."""
    async with httpx.AsyncClient() as client:
        # Fetch available languages first
        response = await client.get(BASE)
        response.raise_for_status()
        languages = re.findall(r'class="list-group-item">(.*?)</a></li>', response.text)

        # Gather data for all languages concurrently
        tasks = [_web_scrape(lang, client) for lang in languages]
        all_rows = await asyncio.gather(*tasks)

    # Flatten the list of lists and create the DataFrame
    df = pd.DataFrame([row for sublist in all_rows for row in sublist])
    return df

def save_df(df: pd.DataFrame, fbasename:str):
    """Saves the DataFrame to a CSV file."""
    df.to_csv(fbasename+".csv", index=False)

    languages_df = df[["language"]].drop_duplicates().reset_index(drop=True)
    languages_df["language_id"] = languages_df.index + 1
    language_id_mapping = languages_df.set_index("language")["language_id"].to_dict()
    df["language_id"] = df["language"].map(language_id_mapping)
    df = df.drop(columns=["language"])
    # df = df.rename(columns={"language_id":"language"})
    df = df[["language_id", "phrase", "meaning", "voteup", "votedown"]]

    db_file = fbasename+".sqlite.db"
    conn = sqlite3.connect(db_file)

    conn.executescript('''
        PRAGMA foreign_keys = ON;
        CREATE TABLE IF NOT EXISTS languages (
            language_id INTEGER PRIMARY KEY,
            language TEXT UNIQUE
        );
        CREATE TABLE IF NOT EXISTS swear (
            language_id INTEGER,
            phrase TEXT,
            meaning TEXT,
            voteup INTEGER,
            votedown INTEGER,
            FOREIGN KEY (language_id) REFERENCES languages(language_id)
        );
    ''')
    languages_df.to_sql("languages", conn, if_exists="replace", index=False)
    df.to_sql("swear", conn, if_exists="replace", index=False)
    conn.commit()
    conn.close()


def is_running_in_ipython():
    """Checks if the code is running in an IPython environment."""
    try:
        from IPython import get_ipython
        return get_ipython() is not None
    except ImportError:
        return False


if __name__ == "__main__":
    if is_running_in_ipython():
        import nest_asyncio
        nest_asyncio.apply()
    df = asyncio.run(web_to_df())
    print(df)